In [1]:
from risk_targeted_hazard import *
from openquake.baselib.general import BASE183
import os

%run -i "C:\Users\ahul697\Desktop\Research\GitHub_Repos\GNS\nzshm-hazlab\hdf5_postprocessing\parse_metadata"

In [2]:
folder = 'data'

oq_id = 32
file_id = str(Path(folder,f'calc_{oq_id}.hdf5'))

### retrieve hcurves from oq file

In [3]:
data = retrieve_data(file_id)
vs30 = check_vs30(data)

### calculate APoE based design intensities

In [4]:
hazard_rps = np.array([25,50,100,250,500,1000,2500])
data['hazard_design'] = {}
data['hazard_design']['hazard_rps'] = hazard_rps.tolist()

for intensity_type in ['acc','disp']:
    [im_hazard, stats_im_hazard] = calculate_hazard_design_intensities(data,hazard_rps,intensity_type)

    data['hazard_design'][intensity_type] = {}
    data['hazard_design'][intensity_type]['im_hazard'] = im_hazard.tolist()
    data['hazard_design'][intensity_type]['stats_im_hazard'] = stats_im_hazard.tolist()

C:\Users\ahul697\Desktop\Research\GitHub_Repos\GNS\risk-targeted-hazard\risk_targeted_hazard\prepare_design_intensities.py:30: RuntimeWarning: divide by zero encountered in log
  im_hazard[i_site,i_imt,:,i_rlz,0] = np.exp(np.interp(np.log(1/hazard_rps), np.log(np.flip(hcurves_rlzs[i_site,i_imt,:,i_rlz])), np.log(np.flip(imtls[imt]))))
C:\Users\ahul697\Desktop\Research\GitHub_Repos\GNS\risk-targeted-hazard\risk_targeted_hazard\prepare_design_intensities.py:42: RuntimeWarning: divide by zero encountered in log
  stats_im_hazard[i_site,i_imt,:,i_stat] = np.exp(np.interp(np.log(1/hazard_rps), np.log(np.flip(hcurves_stats[i_site,i_imt,:,i_stat])), np.log(np.flip(imtls[imt]))))


### get the metadata

In [5]:
source_lt, gsim_lt_dict, rlz_lt = parse_logic_tree_branches(file_id)

C:\Users\ahul697\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\ahul697\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
C:\Users\ahul697\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

### save data

In [6]:
export_name = 'test-export'

parameters_for_exporting = {}
parameters_for_exporting['sites'] = ['Wellington','Christchurch','Dunedin','Auckland']
parameters_for_exporting['imts']  = [imt_from_period(period) for period in [0,0.2,0.5,1.0,2.0,3.0,5.0]]
parameters_for_exporting['rps']   = [25,50,100,250,500,1000,2500]


In [7]:
filename = str(Path(export_name,f'{export_name}_postprocessed_vs30-{vs30}.hdf5'))

# create the initial file with the metadata
with h5py.File(filename,'w') as hf: 
    grp = hf.create_group('metadata')
    # store the intensity measure types and levels for acceleration and displacement-based intensities
    for intensity_type in ['acc','disp']:
        grp.attrs[f'{intensity_type}_imtls'] = str(data['metadata'][f'{intensity_type}_imtls'])
    # store the list of quantiles in the summary statistics
    grp.attrs['quantiles'] = data['metadata']['quantiles']
    # store the realization weights
    grp.attrs['rlz_weights'] = data['metadata']['rlz_weights']
    # store the vs30 of all the sites
    grp.attrs['vs30'] = vs30
    
# add sites to metadata
pd.DataFrame(data['metadata']['sites']).to_hdf(filename,'metadata/sites','a')

# add logic trees to metadata
pd.DataFrame(source_lt).to_hdf(filename,'metadata/logic_trees/source_lt','a')
pd.DataFrame(rlz_lt).to_hdf(filename,'metadata/logic_trees/full_lt','a')
for i,df in gsim_lt_dict.items():
    trt = np.unique(df['trt'].values)[0].replace(' ','_')
    pd.DataFrame(df).to_hdf(filename,f'metadata/logic_trees/gsim_lts/{trt}','a')
    
# add hazard curve data
with h5py.File(filename,'a') as hf: 
    grp = hf.create_group('hcurves')
    for label in ['hcurves_rlzs','hcurves_stats']:
        dset = grp.create_dataset(label,data=np.array(data['hcurves'][label]))


In [8]:
create_metadata_file(export_name,vs30,rlz_lt,gsim_lt_dict)
create_hcurves_single_vs30(export_name,vs30,parameters_for_exporting,data)
create_uhs_single_vs30(export_name,vs30,parameters_for_exporting,data)